# Environment setup

In [2]:
import json
import os 
from dotenv import load_dotenv
from app.controller.utils import to_markdown

# from langchain_community.tools.tavily_search import TavilySearchResults
# from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_core.output_parsers import StrOutputParser

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [3]:
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Experiment

## Prompting design

In [4]:


workout_plans = """
As a Personal Trainer, you are tasked to generate a comprehensive week workout plan that is specifically designed to help the client achieve a particular {fitness_goal}. The plan should include a variety of exercises, targeting different muscle groups, to ensure a balanced and effective workout routine. It should also consider the client's current fitness level, specific needs, and available fitness equipment. The plan must provide clear instructions for each exercise, including the number of sets, repetitions, and rest periods. Also, it should include a weekly schedule with recommended days for each workout and rest days. Lastly, provide tips on proper form and safety measures to avoid injuries and ensure the effectiveness of each exercise.
"""
# Taste Design
instruction = """
You should: 
+ write your answers in bullet points. 
+ write your answers in plain text format. 
+ give a motivation for me. 
+ remind me to rest reasonably.
+ be friendly as much as posible.
"""


my_prompt = workout_plans + instruction
b2m_prompt = PromptTemplate.from_template(my_prompt)

In [5]:
print(b2m_prompt.template)


As a Personal Trainer, you are tasked to generate a comprehensive week workout plan that is specifically designed to help the client achieve a particular {fitness_goal}. The plan should include a variety of exercises, targeting different muscle groups, to ensure a balanced and effective workout routine. It should also consider the client's current fitness level, specific needs, and available fitness equipment. The plan must provide clear instructions for each exercise, including the number of sets, repetitions, and rest periods. Also, it should include a weekly schedule with recommended days for each workout and rest days. Lastly, provide tips on proper form and safety measures to avoid injuries and ensure the effectiveness of each exercise.

You should: 
+ write your answers in bullet points. 
+ write your answers in plain text format. 
+ give a motivation for me. 
+ remind me to rest reasonably.
+ be friendly as much as posible.



## Create Agent

In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GEMINI_API_KEY, 
                             temperature=0.7, top_p=0.85, top_k=3)

output_parser = StrOutputParser()

b2m_chain = LLMChain(llm=llm, prompt=b2m_prompt, verbose=True)

In [7]:
b2m_chain = b2m_prompt | \
            llm | \
            output_parser

## Inference

In [8]:
response = b2m_chain.invoke({'fitness_goal': "I want to have 6 packs. In addition, I also want huge back and shoulder."})

In [10]:
to_markdown(response)

> **Motivation:**
> 
> "Unlocking your dream physique is within your grasp! With dedication and consistency, this tailored workout plan will sculpt your core, back, and shoulders into the definition you desire."
> 
> **Workout Plan:**
> 
> **Day 1: Chest and Triceps**
> 
> * Barbell Bench Press: 3 sets of 8-12 reps
> * Incline Dumbbell Press: 3 sets of 10-15 reps
> * Triceps Pushdowns: 3 sets of 12-15 reps
> * Overhead Triceps Extensions: 3 sets of 10-15 reps
> 
> **Day 2: Back and Biceps**
> 
> * Pull-Ups: 3 sets of 8-12 reps
> * Barbell Rows: 3 sets of 10-15 reps
> * Hammer Curls: 3 sets of 12-15 reps
> * Concentration Curls: 3 sets of 10-15 reps
> 
> **Day 3: Rest**
> 
> **Day 4: Legs**
> 
> * Barbell Squats: 3 sets of 8-12 reps
> * Leg Press: 3 sets of 10-15 reps
> * Hamstring Curls: 3 sets of 12-15 reps
> * Calf Raises: 3 sets of 15-20 reps
> 
> **Day 5: Shoulders and Abs**
> 
> * Overhead Press: 3 sets of 8-12 reps
> * Lateral Raises: 3 sets of 10-15 reps
> * Front Raises: 3 sets of 12-15 reps
> * Plank: Hold for 30-60 seconds, 3 sets
> 
> **Day 6: Rest**
> 
> **Day 7: Cardio and Core**
> 
> * Running or Cycling: 30 minutes at moderate intensity
> * Russian Twists: 3 sets of 20-30 reps
> * Leg Raises: 3 sets of 15-20 reps
> 
> **Tips for Proper Form and Safety:**
> 
> * Maintain a neutral spine and engage your core throughout all exercises.
> * Choose weights that challenge you while allowing for proper form.
> * Focus on the mind-muscle connection and control the movement.
> * Rest adequately between sets to allow for muscle recovery.
> * Listen to your body and take rest days when needed.
> * Consult a medical professional if you experience any pain or discomfort.
> 
> **Remember:**
> 
> Rest is crucial for muscle recovery and growth. Aim for 7-9 hours of quality sleep each night. Stay hydrated by drinking plenty of water throughout the day. Fuel your body with a nutritious diet to support your training goals. Consistency is key, so stick to your workout schedule as much as possible. With hard work and dedication, you will achieve the physique you desire.

In [1]:
from controller import fitness
from schema import GetWorkoutPlan

fitness_goal = GetWorkoutPlan(fitness_goal="I want to be a body-builder")
workout_plans = fitness.generate_workout_plan(fitness_goal=fitness_goal, instruction=None)
workout_plans

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised DeadlineExceeded: 504 Deadline Exceeded.


"**Hey there, future bodybuilder!**\n\nI'm excited to help you craft a comprehensive workout plan that will unleash your inner beast and help you achieve your bodybuilding goals. Let's dive right in!\n\n**Workout Plan:**\n\n**Day 1: Chest and Triceps**\n\n* Barbell bench press: 3 sets of 8-12 reps\n* Incline dumbbell press: 3 sets of 8-12 reps\n* Dumbbell flyes: 3 sets of 10-15 reps\n* Triceps pushdowns: 3 sets of 10-15 reps\n* Skullcrushers: 3 sets of 10-15 reps\n\n**Day 2: Back and Biceps**\n\n* Barbell row: 3 sets of 8-12 reps\n* Pull-ups: 3 sets of 8-12 reps\n* Lat pulldowns: 3 sets of 10-15 reps\n* Bicep curls: 3 sets of 10-15 reps\n* Hammer curls: 3 sets of 10-15 reps\n\n**Day 3: Legs (Quadriceps)**\n\n* Barbell squats: 3 sets of 8-12 reps\n* Leg press: 3 sets of 8-12 reps\n* Leg extensions: 3 sets of 10-15 reps\n* Calf raises: 3 sets of 15-20 reps\n\n**Day 4: Legs (Hamstrings and Glutes)**\n\n* Romanian deadlifts: 3 sets of 8-12 reps\n* Hamstring curls: 3 sets of 10-15 reps\n* G